In [1]:
from parameters import ENG_STOPWORDS, N_GRAM
from helpers import treat_article, compute_plagiarism_from_articles, compute_many_to_many_sim


In [6]:
### File paths 
fr_path = "./txt files/french.txt"
en_path = "./txt files/english.txt"
it_path = "./txt files/italian.txt"
es_path = "./txt files/spanish.txt"

fr_article = treat_article(fr_path,ENG_STOPWORDS,N_GRAM)
en_article = treat_article(en_path,ENG_STOPWORDS,N_GRAM)
it_article = treat_article(it_path,ENG_STOPWORDS,N_GRAM)
es_article = treat_article(es_path,ENG_STOPWORDS,N_GRAM)

france_lat_lon = (46.00,2.00)
spain_lat_lon = (40.46,-3.75)
italy_lat_lon = (42.23,13.57)
england_lat_lon = (52.35,-1.17)

languages = ["french","english","italian","spanish"]
articles = [fr_article,en_article,it_article,es_article]
lat_lon_values = [france_lat_lon,england_lat_lon,italy_lat_lon,spain_lat_lon]

In [5]:
viewer , score  = compute_plagiarism_from_articles(fr_article, es_article)
viewer(60)

The two articles have a similarity score of 3.98, with 124 matching n-gram. You can use the viewer to visualize the matching sequences
was augmented by his successors and these companions of his soil and of the genius of his inhabitants geniuses had also also the name name of parliament They are The population totality of the kingdom which remains today thirteen in number Paris Toulouse feels always
or other superior Courts similar to that of Paris The number was increased by his successors and these Courts covered also also the name name of Parliament There are currently thirteen Paris Toulouse Grenoble Bordeaux Dijon Rouen Aix Rennes Pau Metz Douai Besanzon and Nanci


In [7]:
compute_many_to_many_sim(articles, languages, True, lat_lon_values)

           french   english   italian spanish
french        NaN       NaN       NaN     NaN
english  0.015613       NaN       NaN     NaN
italian   2.35021  0.010275       NaN     NaN
spanish  3.982656  0.015681  2.973536     NaN
